In [4]:
!pip3 install Cython

In [5]:
!pip3 install pycocotools

  Using cached pycocotools-2.0.0.tar.gz (1.5 MB)
  Created wheel for pycocotools: filename=pycocotools-2.0.0-cp37-cp37m-linux_x86_64.whl size=279275 sha256=65c5bb7de491b9ecf02c56565b55e8bbb2e9b68fd132130c677bb5259b6413db
  Stored in directory: /home/kimsoohyun/.cache/pip/wheels/4d/50/dc/e1f07e9eb5678a0ee21bc091220f1f3806ba8e48ef3f2083cb
Successfully built pycocotools


In [29]:
!pip3 install labelImg

     |████████████████████████████████| 248 kB 775 kB/s eta 0:00:01
     |████████████████████████████████| 5.7 MB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 63.6 MB 292 kB/s eta 0:00:01    |███▍                            | 6.7 MB 1.4 MB/s eta 0:00:41     |███████████████████▎            | 38.3 MB 2.2 MB/s eta 0:00:12     |███████████████████████▎        | 46.3 MB 2.8 MB/s eta 0:00:07
     |████████████████████████████████| 253 kB 776 kB/s eta 0:00:01


In [10]:
%%bash
cd dataset/03-models/ob_model/models/research
pip3 install .

Processing /home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/03-models/ob_model/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1017522 sha256=f363291418b28a65b338da9ba3d4e6e5d9f26badf2fa290242a00e552f49f1ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-ywbjwq8x/wheels/c3/37/13/1e3e810734edbd3864f6281436173fbbff2fe5451d511b02ac
Successfully built object-detection


In [20]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import pathlib

In [21]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [22]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [23]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file, untar=True)

    model_dir = pathlib.Path(model_dir)/"saved_model"

    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

In [24]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/03-models/ob_model/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [26]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/03-models/ob_model/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[PosixPath('/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/03-models/ob_model/models/research/object_detection/test_images/image1.jpg'),
 PosixPath('/home/kimsoohyun/00-Research/02-Graph/02-image_detection/04-clickable/dataset/03-models/ob_model/models/research/object_detection/test_images/image2.jpg')]

In [28]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
class_names = sorted(train_data_gen.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
print('Classes: {}'.format(class_names))

## get result labels
predicted_batch = model.predict(image_batch_test)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

label_id = np.argmax(label_batch_test, axis=-1)

## plot
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
    plt.subplot(6, 5, n+1)
    plt.imshow(image_batch_test[n])
    color = 'green' if predicted_id[n] == label_id[n] else 'red'
    plt.title(predicted_label_batch[n].title(), color=color)
    plt.axis('off')
_ = plt.suptitle('Model predictions (green: correct, red: incorrect)')